Математическая модель метрики галлюцинации, реализованная в коде, выглядит следующим образом:

# Обозначения:

- $S_{h}$ — итоговый **hallucination score** (метрика галлюцинации).
- $S_c$ — **concept score** (концептуальная согласованность), вычисляемая через **косинусное сходство** между эмбеддингами ответа модели и контекста.
- $F_h$ — количество **галлюцинированных фактов** (факты в ответе модели, которых нет в контексте).
- $F_m$ — количество **пропущенных фактов** (факты, присутствующие в контексте, но отсутствующие в ответе модели).
- $F_{total}$ — общее число фактов в ответе модели.
- $w_c$ = 0.4 — вес концептуальной согласованности.
- $w_f$ = 0.6 — вес фактологической ошибки.
---

## 1. Концептуальная согласованность

$
S_c = \cos(\theta) = \frac{V_{model} \cdot V_{context}}{|V_{model}| |V_{context}|}
$

где \( $V_{model}$ \) и \( $V_{context}$ \) — это BERT-эмбеддинги CLS-токенов.

---

## 2. Доля фактологических ошибок
$
E_f =
\begin{cases}
1, & \text{если } F_{total} = 0 \text{ и } |F_m| > 0 \\
0, & \text{если } F_{total} = 0 \text{ и } |F_m| = 0 \\
\frac{|F_h| + |F_m|}{F_{total}}, & \text{иначе}
\end{cases}$

где $E_f$ — **fact error ratio** (доля ошибок в фактах).

---

## 3. Итоговая метрика галлюцинации
$
S_h = w_c (1 - S_c) + w_f E_f
$

где:

- Чем выше $S_h$, тем больше модель галлюцинирует.
- Чем ниже $S_h$, тем качественнее ответ модели.
- $S_h$ нормализуется в пределах $[0, 1]$:

$
S_h = \min(1, \max(0, S_h))
$


In [ ]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 2.1 MB/s eta 0:00:00


In [4]:
import torch
import re
from transformers import BertTokenizer, BertModel

class FactVerificationScore:
    def __init__(self, model_name='bert-base-cased'):
        self.tokenizer = BertTokenizer.from_pretrained(model_name)
        self.model = BertModel.from_pretrained(model_name)
        self.model.eval()
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        self.model.to(self.device)

    def _encode_text(self, text):
        """Кодирует текст в эмбеддинги с помощью BERT (используется только при необходимости)"""
        inputs = self.tokenizer(text, padding=True, truncation=True, max_length=512, return_tensors="pt")
        inputs = {k: v.to(self.device) for k, v in inputs.items()}
        with torch.no_grad():
            outputs = self.model(**inputs)
            embeddings = outputs.last_hidden_state[:, 0, :].cpu().numpy()  # CLS-токен
        return embeddings

    def _extract_facts(self, text):
        """Извлекает факты: числа, даты, имена собственные, включая сокращения"""
        patterns = [
            r'\b(?:НИУ )?ВШЭ\b',  # ВШЭ или НИУ ВШЭ
            r'\b\d{4}\b',          # Годы
            r'[А-ЯA-Z][а-яa-z]+(?: [А-ЯA-Z][а-яa-z]+)*'  # Имена собственные
        ]
        facts = set()
        for pattern in patterns:
            facts.update(re.findall(pattern, text))
        return facts

    def _check_contradictions(self, model_facts, context_facts):
        """Проверяет наличие противоречий между фактами"""
        contradictions = 0
        for model_fact in model_facts:
            if model_fact in context_facts:
                continue  # Факт совпадает, не противоречие
            # Если факт из model_output — год, а в context есть другой год
            if re.match(r'\b\d{4}\b', model_fact):
                context_years = {f for f in context_facts if re.match(r'\b\d{4}\b', f)}
                if context_years and model_fact not in context_years:
                    contradictions += 1
            # Если это сущность (не год), и её нет в контексте — считаем потенциальным противоречием
            elif model_fact not in context_facts:
                contradictions += 1
        return contradictions

    def calculate_score(self, model_output, context):
        # 1. Извлечение фактов
        model_facts = self._extract_facts(model_output)
        context_facts = self._extract_facts(context)

        # 2. Проверка противоречий
        contradictions = self._check_contradictions(model_facts, context_facts)

        # 3. Оценка пропущенных фактов (менее критична, чем противоречия)
        missing_facts = len(context_facts - model_facts)

        # 4. Расчет метрики
        total_model_facts = len(model_facts) if len(model_facts) > 0 else 1  # Избегаем деления на 0
        contradiction_ratio = contradictions / total_model_facts  # Доля противоречий
        missing_fact_penalty = min(0.5, missing_facts / (len(context_facts) + 1))  # Штраф за пропуски, ограничен 0.5

        # Итоговая метрика: больше вес у противоречий, меньше у пропущенных фактов
        w_contradiction = 0.8  # Высокий вес для противоречий
        w_missing = 0.2        # Низкий вес для пропущенных фактов

        hallucination_score = (
            w_contradiction * contradiction_ratio +
            w_missing * missing_fact_penalty
        )
        hallucination_score = min(1.0, max(0.0, hallucination_score))  # Ограничение [0, 1]

        # Интерпретация результата
        interpretation = (
            "Корректный ответ" if hallucination_score < 0.3 else
            "Скорее корректный" if hallucination_score < 0.5 else
            "Скорее галлюцинация" if hallucination_score < 0.7 else
            "Галлюцинация"
        )

        return {
            "hallucination_score": hallucination_score,
            "contradictions": contradictions,
            "missing_facts": missing_facts,
            "contradiction_ratio": contradiction_ratio,
            "missing_fact_penalty": missing_fact_penalty,
            "interpretation": interpretation
        }

# Пример использования
if __name__ == "__main__":
    evaluator = FactVerificationScore()

    # Пример 1: Противоречие в годе
    model_output = "ВШЭ - это крупнейший университет России, основанный в 1990 году."
    context = "НИУ ВШЭ был основан в 1992 году. Высшая школа экономики является одним из ведущих университетов России."
    result = evaluator.calculate_score(model_output, context)
    print("Пример 1 (противоречие в годе):")
    print(f"Метрика галлюцинации: {result['hallucination_score']:.3f}")
    print(f"Количество противоречий: {result['contradictions']}")
    print(f"Количество пропущенных фактов: {result['missing_facts']}")
    print(f"Доля противоречий: {result['contradiction_ratio']:.3f}")
    print(f"Штраф за пропущенные факты: {result['missing_fact_penalty']:.3f}")
    print(f"Интерпретация: {result['interpretation']}\n")

    # Пример 2: Нет противоречий
    model_output_1 = "Для получения социальной стипендии необходимо подать заявку через сервис единого окна в модуле LMS, прикрепив отсканированные документы (личное заявление и документ, подтверждающий льготу)."
    context_1 = "Дополнительные документы для получения социальной стипендии нужно предоставить в Центр стипендиальных и благотворительных программ НИУ ВШЭ или через сервис единого окна в модуле LMS. Это включает копию заявления и копию действующей справки МСЭ.\
    Также Для получения социальной стипендии необходимо подать заявку через сервис единого окна в модуле LMS, прикрепив отсканированные документы (личное заявление и документ, подтверждающий льготу)"
    result = evaluator.calculate_score(model_output_1, context_1)
    print("Пример 2 (нет противоречий):")
    print(f"Метрика галлюцинации: {result['hallucination_score']:.3f}")
    print(f"Количество противоречий: {result['contradictions']}")
    print(f"Количество пропущенных фактов: {result['missing_facts']}")
    print(f"Доля противоречий: {result['contradiction_ratio']:.3f}")
    print(f"Штраф за пропущенные факты: {result['missing_fact_penalty']:.3f}")
    print(f"Интерпретация: {result['interpretation']}")

Пример 1 (противоречие в годе):
Метрика галлюцинации: 0.633
Количество противоречий: 2
Количество пропущенных фактов: 3
Доля противоречий: 0.667
Штраф за пропущенные факты: 0.500
Интерпретация: Скорее галлюцинация

Пример 2 (нет противоречий):
Метрика галлюцинации: 0.900
Количество противоречий: 1
Количество пропущенных фактов: 5
Доля противоречий: 1.000
Штраф за пропущенные факты: 0.500
Интерпретация: Галлюцинация


In [6]:
!pip install numpy nltk

In [25]:
from typing import List
import numpy as np
import nltk
from nltk.translate import bleu_score
nltk.download('punkt')

def hallucination_score(contexts: List[str], model_output: str) -> float:
    """
    Оценивает потенциальные галлюцинации в ответе LLM, сравнивая его с набором контекстов.
    Использует BLEU (precision) для измерения n-граммного соответствия между model_output и contexts.

    Args:
        contexts (List[str]): Список строк контекста, на основе которых модель должна была генерировать ответ.
        model_output (str): Ответ, сгенерированный LLM, который нужно проверить.

    Returns:
        float: Средний BLEU-score (precision для биграмм) по всем контекстам.
               Низкое значение (<0.3) может указывать на галлюцинацию.
    """
    bleu_scores = []

    # Проходим по каждому контексту
    for context in contexts:
        try:
            # Вычисляем BLEU с учетом биграмм (max_order=2)
            score = bleu_score.sentence_bleu(
                references=[context.split()],  # Разбиваем контекст на токены
                hypothesis=model_output.split(),  # Разбиваем ответ модели на токены
                weights=(0, 1, 0, 0)  # Используем только биграммы (precision2)
            )
            bleu_scores.append(score)
        except ZeroDivisionError:
            # Если деление на ноль (например, пустой контекст или ответ), добавляем 0
            bleu_scores.append(0)

    # Возвращаем среднее значение BLEU по всем контекстам
    return np.mean(bleu_scores) if bleu_scores else 0.0

# Пример использования
if __name__ == "__main__":
    # Пример данных
    contexts = [
        "Для оформления материальной помощи на оплату общежития необходимо предоставить определенные медицинские документы, такие как справка по форме 086у или сертификат МОДФ.",
        "Также нужно учесть, что условия предоставления мест и размещения в общежитиях различаются."
         "Более подробную информацию можно получить в Дирекции по управлению общежитиями, гостиницами, учебно-оздоровительными комплексами."
    ]
    model_output_good = "Для оформления материальной помощи на оплату общежития необходимо предоставить определенные медицинские документы"
    model_output_bad = "Интернет изобрел Илон Маск в 2010 году в гараже."

    # Проверка хорошего ответа
    good_score = hallucination_score(contexts, model_output_good)
    print(f"Score для хорошего ответа: {good_score:.4f}")

    # Проверка плохого (галлюцинирующего) ответа
    bad_score = hallucination_score(contexts, model_output_bad)
    print(f"Score для плохого ответа: {bad_score:.4f}")

Score для хорошего ответа: 0.2147
Score для плохого ответа: 0.0000


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
/usr/local/lib/python3.11/dist-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.11/dist-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.11/dist-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram over

3. Анализ ключевых слов
Идея: Извлечь ключевые слова из contexts и проверить их присутствие в model_output. Отсутствие важных терминов может указывать на отклонение от темы.

Реализация:

In [61]:
from nltk.corpus import stopwords
nltk.download('stopwords')
from collections import Counter

def extract_keywords(text: str, stop_words) -> set:
    words = text.lower().split()
    return {word for word in words if word not in stop_words and len(word) > 3}

def hallucination_score(contexts: List[str], model_output: str) -> float:
    bleu_scores = []
    stop_words = set(stopwords.words('russian'))  # Для русского языка

    # Извлекаем ключевые слова из всех контекстов
    context_keywords = set()
    for context in contexts:
        context_keywords.update(extract_keywords(context, stop_words))

    # Ключевые слова из model_output
    output_keywords = extract_keywords(model_output, stop_words)

    # Доля пересечения ключевых слов
    keyword_overlap = len(context_keywords & output_keywords) / len(context_keywords) if context_keywords else 0.0

    for context in contexts:
        try:
            score = bleu_score.sentence_bleu(
                references=[context.split()],
                hypothesis=model_output.split(),
                weights=(0, 1, 0, 0)
            )
            bleu_scores.append(score)
        except ZeroDivisionError:
            bleu_scores.append(0)

    bleu_mean = np.mean(bleu_scores) if bleu_scores else 0.0

    # Комбинируем BLEU и пересечение ключевых слов
    return 0.7 * bleu_mean + 0.3 * keyword_overlap

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# Самая лучшая
---
### Математическая модель метрики "степень галлюцинации"

#### Входные данные:
- $C$ = ${c_1, c_2, ..., c_n}$ — список контекстов (строк), где $n$ — количество контекстов.
- $O$ — строка, представляющая вывод модели (ответ).
- $S$ — множество стоп-слов (на русском языке).

#### Шаг 1: Извлечение ключевых слов
Для каждой строки $x$ (контекста или вывода модели) определяется множество ключевых слов:
$$
K(x) = \{ w \mid w \in x.split(), w \notin S, len(w) > 3 \}
$$
- $w$ — слово в нижнем регистре.
- Условие $$w \notin S$$ исключает стоп-слова.
- Условие $len(w) > 3$ исключает короткие слова.

Множества ключевых слов:
- $K_C = \bigcup_{i=1}^n K(c_i)$ — объединение ключевых слов всех контекстов.
- $K_O = K(O)$ — ключевые слова из вывода модели.

#### Шаг 2: Jaccard-подобная схожесть ключевых слов
Схожесть между контекстом и выводом по ключевым словам вычисляется как:
$$
J = \frac{|K_C \cap K_O|}{\max(|K_O|, 1)}
$$
- $|K_C \cap K_O|$ — количество общих ключевых слов.
- $max(|K_O|, 1)$ — нормализация по количеству слов в выводе (избегаем деления на 0).
- Это отражает долю пересечения ключевых слов, избегая чрезмерного штрафа за краткость ответа.

#### Шаг 3: BLEU-оценка
Для каждого контекста $c_i$ вычисляется BLEU-оценка между $c_i$ и $O$:
$$
B_i = BLEU(c_i.split(), O.split(), w)
$$
- $w = (0.7, 0.3, 0, 0)$ — веса для униграмм $(70%)$ и биграмм $(30%)$, триграммы и выше не учитываются.
- Если возникает $ZeroDivisionError,B_i = 0$.

Средняя BLEU-оценка по всем контекстам:
$
B = \frac{1}{n} \sum_{i=1}^n B_i \quad \text{(или 0, если } n = 0\text{)}
$

#### Шаг 4: Штраф за неожиданные слова
Неожиданные слова — это слова в выводе, отсутствующие в контексте:
$
U = K_O \setminus K_C
$
Штраф за неожиданные слова:
$$
P = \begin{cases}
\frac{|U|}{|K_O| + \epsilon} & \text{если } U \neq \emptyset, \\
0 & \text{если } U = \emptyset,
\end{cases}
$$
- $epsilon = 10^{-6}$ — малая константа для избежания деления на 0.
- Штраф пропорционален доле неожиданных слов в выводе.

#### Шаг 5: Итоговый коэффициент
Итоговая метрика "степень галлюцинации" (точнее, степень правдоподобности) вычисляется как:
$
H = \left( 0.6 \cdot B + 0.4 \cdot J \right) \cdot (1 - P)
$
- $0.6 \cdot B$ — вклад BLEU-оценки (60%).
- $0.4 \cdot J$ — вклад пересечения ключевых слов (40%).
- $(1 - P)$ — множитель, уменьшающий итоговую оценку при наличии неожиданных слов.

Ограничение на отрицательные значения:
$
H_{final} = \max(H, 0)
$

---

### Интерпретация
- $H_{final} \in [0, 1]$:
  - $H_{final} \approx 1$: Вывод модели полностью соответствует контексту (высокий BLEU, большое пересечение ключевых слов, нет неожиданных слов).
  - $H_{final} \approx 0$: Вывод модели сильно отклоняется от контекста (низкий BLEU, мало общих слов, много неожиданных слов).
- Чем выше $H_{final}$, тем меньше "галлюцинаций" в ответе.

---

### Пример
#### Вход:
- $C = ["Москва - столица России"]$
- $O = "Москва - большой город"$
- $S = \{"и", "в", "на"\}$ (упрощённый набор стоп-слов)

#### Вычисления:

$$
1. K_C = { "москва", "столица", "россии" }
$$$$
2. K_O = { "москва", "большой", "город"}
$$$$
3. J = \frac{|\{ "москва" \}|}{\max(3, 1)} = \frac{1}{3} \approx 0.333)
$$$$
4. B —  BLEU-оценка между "москва столица россии" и "москва большой город" (допустим, ( B \approx 0.4 )).
$$$$
5. U = { "большой", "город" },  ( P = \frac{2}{3 + 10^{-6}} \approx 0.667 )
$$$$
6. (H = (0.6 \cdot 0.4 + 0.4 \cdot 0.333) \cdot (1 - 0.667) = (0.24 + 0.133) \cdot 0.333 \approx 0.373 \cdot 0.333 \approx 0.124)
$$$$
7. H_{final} = 0.124
$$
#### Результат:
$H_{final} = 0.124$ — низкий показатель, указывающий на наличие галлюцинаций ("большой город" не из контекста).

In [ ]:
!pip install nltk numpy scikit-learn sentence-transformers

In [71]:
import nltk
import numpy as np
from nltk.translate import bleu_score
from nltk.corpus import stopwords
from collections import Counter
from typing import List

nltk.download('stopwords')

def extract_keywords(text: str, stop_words) -> set:
    """Извлекает ключевые слова, исключая стоп-слова и короткие токены."""
    words = text.lower().split()
    return {word for word in words if word not in stop_words and len(word) > 3}

def hallucination_score(contexts: List[str], model_output: str) -> float:
    """Оценивает степень галлюцинации ответа модели по сравнению с контекстом."""

    stop_words = set(stopwords.words('russian'))  # Стоп-слова для русского языка

    # Извлекаем ключевые слова из всех контекстов
    context_keywords = set()
    for context in contexts:
        context_keywords.update(extract_keywords(context, stop_words))

    # Ключевые слова из ответа модели
    output_keywords = extract_keywords(model_output, stop_words)

    # Jaccard similarity (не штрафуем, если ответ короче, но точен)
    keyword_overlap = len(context_keywords & output_keywords) / max(len(output_keywords), 1)

    # BLEU-оценка (с униграммами и биграммами)
    bleu_scores = []
    for context in contexts:
        try:
            score = bleu_score.sentence_bleu(
                references=[context.split()],
                hypothesis=model_output.split(),
                weights=(0.7, 0.3, 0, 0)  # Униграммы (70%) и биграммы (30%)
            )
            bleu_scores.append(score)
        except ZeroDivisionError:
            bleu_scores.append(0)

    bleu_mean = np.mean(bleu_scores) if bleu_scores else 0.0

    # Штраф за неожиданные слова (только если они действительно выбиваются из контекста)
    unexpected_words = output_keywords - context_keywords
    if unexpected_words:
        unexpected_penalty = len(unexpected_words) / (len(output_keywords) + 1e-6)
    else:
        unexpected_penalty = 0  # Если нет неожиданных слов, штрафа нет

    # Итоговый коэффициент (BLEU + семантическое пересечение - штраф за ошибки)
    final_score = (0.6 * bleu_mean + 0.4 * keyword_overlap) * (1 - unexpected_penalty)

    return max(final_score, 0)  # Исключаем отрицательные значения


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Этот код предназначен для оценки степени галлюцинации (т. е. отклонения ответа модели от контекста). Он использует несколько методов анализа текста:

1. Извлечение ключевых слов (extract_keywords)
Использует nltk.word_tokenize для разбиения текста на токены.
Исключает стоп-слова (stopwords.words('russian')).
Учитывает только слова длиной > 3 символов.
Возвращает Counter, содержащий частоту появления слов.
2. Косинусная схожесть (cosine_similarity)
Вычисляет сходство между двумя наборами ключевых слов, представляя их в виде векторных представлений.
Использует скалярное произведение и нормализацию, чтобы получить коэффициент от 0 до 1.
3. Семантическая схожесть (semantic_similarity)
Преобразует контексты и ответ модели в векторные представления с помощью SentenceTransformer (distiluse-base-multilingual-cased).
Вычисляет среднюю косинусную схожесть между векторами контекстов и вектором ответа.
4. BLEU-оценка (bleu_score)
Оценивает совпадение между model_output и каждым контекстом на уровне n-грамм.
Использует веса (0,5 для униграмм, 0,3 для биграмм, 0,2 для триграмм), чтобы учитывать разные уровни совпадений.
5. Штраф за неожиданные слова
Определяет слова, которые есть в ответе, но отсутствуют в контексте.
Увеличивает штраф до 0,8, если таких слов слишком много.

Этот метод комплексно оценивает галлюцинации, балансируя между точностью (BLEU), смысловым соответствием (SentenceTransformer) и лексическим пересечением (ключевые слова). Если ответ сильно отклоняется от контекста, он получит низкий балл, а если близок к контексту — высокий.

In [68]:
import nltk
import numpy as np
from nltk.translate import bleu_score
from nltk.corpus import stopwords
from collections import Counter
from typing import List
from sklearn.feature_extraction.text import TfidfVectorizer
from sentence_transformers import SentenceTransformer, util

nltk.download('stopwords')
nltk.download('punkt')

# Используем предобученную модель для семантического сравнения
model = SentenceTransformer('distiluse-base-multilingual-cased')

def extract_keywords(text: str, stop_words) -> Counter:
    """Извлекает ключевые слова и их частоту."""
    words = [word for word in nltk.word_tokenize(text.lower()) if word not in stop_words and len(word) > 3]
    return Counter(words)

def cosine_similarity(counter1: Counter, counter2: Counter) -> float:
    """Взвешенная косинусная схожесть между двумя наборами ключевых слов."""
    all_words = set(counter1.keys()).union(counter2.keys())
    if not all_words:
        return 0.0

    vec1 = np.array([counter1[word] for word in all_words])
    vec2 = np.array([counter2[word] for word in all_words])

    norm1 = np.linalg.norm(vec1) + 1e-6
    norm2 = np.linalg.norm(vec2) + 1e-6

    return np.dot(vec1, vec2) / (norm1 * norm2)

def semantic_similarity(contexts: List[str], model_output: str) -> float:
    """Оценка семантической схожести с использованием SentenceTransformer."""
    context_embeddings = model.encode(contexts, convert_to_tensor=True)
    output_embedding = model.encode(model_output, convert_to_tensor=True)
    similarity_scores = util.pytorch_cos_sim(context_embeddings, output_embedding)
    return max(similarity_scores.mean().item(), 0.0)

def hallucination_score(contexts: List[str], model_output: str) -> float:
    """Оценка галлюцинации с учётом семантической близости и неожиданных слов."""
    stop_words = set(stopwords.words('russian'))

    # Объединяем все контексты и извлекаем ключевые слова
    context_text = " ".join(contexts)
    context_keywords = extract_keywords(context_text, stop_words)
    output_keywords = extract_keywords(model_output, stop_words)

    # Косинусная схожесть
    keyword_similarity = cosine_similarity(context_keywords, output_keywords)

    # Семантическая схожесть с SentenceTransformer
    semantic_score = semantic_similarity(contexts, model_output)

    # BLEU-оценка
    bleu_scores = []
    for context in contexts:
        try:
            score = bleu_score.sentence_bleu(
                references=[nltk.word_tokenize(context.lower())],
                hypothesis=nltk.word_tokenize(model_output.lower()),
                weights=(0.5, 0.3, 0.2, 0)  # Униграммы (50%), биграммы (30%), триграммы (20%)
            )
            bleu_scores.append(score)
        except:
            bleu_scores.append(0)
    bleu_mean = np.mean(bleu_scores) if bleu_scores else 0.0

    # Штраф за неожиданные слова (усиленный)
    unexpected_words = output_keywords - context_keywords
    unexpected_penalty = min(len(unexpected_words) / (len(output_keywords) + 1e-6), 0.8)  # Усиливаем штраф до 0.8

    # Итоговый скор (BLEU + семантическая схожесть + ключевые слова - штраф за неожиданные слова)
    final_score = (0.45 * bleu_mean + 0.25 * semantic_score + 0.3 * keyword_similarity) * (1 - unexpected_penalty)

    print(f"BLEU: {bleu_mean}, Semantic: {semantic_score}, Keyword Sim: {keyword_similarity}, Penalty: {unexpected_penalty}")

    return max(final_score, 0.0)


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [72]:
# Пример использования
if __name__ == "__main__":
    # Пример данных
    contexts = [
        "Для оформления материальной помощи на оплату общежития необходимо предоставить определенные медицинские документы, такие как справка по форме 086у или сертификат МОДФ.",
        "Также нужно учесть, что условия предоставления мест и размещения в общежитиях различаются."
         "Более подробную информацию можно получить в Дирекции по управлению общежитиями, гостиницами, учебно-оздоровительными комплексами."
    ]
    model_output_good = "Для оформления материальной помощи на оплату общежития необходимо предоставить определенные медицинские документы"
    model_output_bad = "Интернет изобрел Илон Маск в 2010 году в гараже."

    # Проверка хорошего ответа
    good_score = hallucination_score(contexts, model_output_good)
    print(f"Score для хорошего ответа: {good_score:.4f}")

    # Проверка плохого (галлюцинирующего) ответа
    bad_score = hallucination_score(contexts, model_output_bad)
    print(f"Score для плохого ответа: {bad_score:.4f}")

Score для хорошего ответа: 0.4406
Score для плохого ответа: 0.0000


In [73]:
# Number 3.
# Пример 1
model_output_1 = "Для продления социальной стипендии необходимо подать заявку через сервис единого окна в модуле LMS, прикрепив отсканированные документы (личное заявление и документ, подтверждающий льготу)."
context_1 = ["Дополнительные документы для продления социальной стипендии нужно предоставить в Центр стипендиальных и благотворительных программ НИУ ВШЭ. Это включает копию заявления и копию действующей справки МСЭ. Для продления социальной стипендии необходимо подать заявку через сервис единого окна в модуле LMS, прикрепив отсканированные документы (личное заявление и документ, подтверждающий льготу)."]
# Проверка хорошего ответа
score = hallucination_score(context_1, model_output_1)
print(f"Score для хорошего ответа: {score:.4f}")

model_output_2 = "Для продления социальной стипендии необходимо подать заявку через сервис единого окна в модуле SMART LMS, прикрепив отсканированные документы (Паспорт и водительские права)."
# Проверка плохого ответа
score = hallucination_score(context_1, model_output_2)
print(f"Score для хорошего ответа: {score:.4f}")

Score для хорошего ответа: 0.6023
Score для хорошего ответа: 0.3544


In [36]:
import nltk
import numpy as np
from nltk.translate import bleu_score
from nltk.corpus import stopwords
from collections import Counter
from typing import List
from sentence_transformers import SentenceTransformer, util

nltk.download('stopwords')

# Загружаем предобученную модель для семантического сходства
semantic_model = SentenceTransformer("distiluse-base-multilingual-cased")

def extract_keywords(text: str, stop_words) -> set:
    """Извлекает ключевые слова, исключая стоп-слова и короткие токены."""
    words = text.lower().split()
    return {word for word in words if word not in stop_words and len(word) > 3}

def hallucination_score(contexts: List[str], model_output: str) -> float:
    """Оценивает степень галлюцинации ответа модели по сравнению с контекстом."""

    stop_words = set(stopwords.words('russian'))  # Стоп-слова для русского языка

    # Извлекаем ключевые слова из всех контекстов
    context_keywords = set()
    for context in contexts:
        context_keywords.update(extract_keywords(context, stop_words))

    # Ключевые слова из ответа модели
    output_keywords = extract_keywords(model_output, stop_words)

    # Jaccard similarity (не штрафуем, если ответ короче, но точен)
    keyword_overlap = len(context_keywords & output_keywords) / max(len(output_keywords), 1)

    # BLEU-оценка (униграммы и биграммы)
    bleu_scores = []
    for context in contexts:
        try:
            score = bleu_score.sentence_bleu(
                references=[context.split()],
                hypothesis=model_output.split(),
                weights=(0.7, 0.3, 0, 0)  # Униграммы (70%) и биграммы (30%)
            )
            bleu_scores.append(score)
        except ZeroDivisionError:
            bleu_scores.append(0)

    bleu_mean = np.mean(bleu_scores) if bleu_scores else 0.0

    # Семантическое сходство (Sentence Transformer)
    context_embeddings = semantic_model.encode(contexts, convert_to_tensor=True)
    output_embedding = semantic_model.encode(model_output, convert_to_tensor=True)

    semantic_similarity = util.cos_sim(output_embedding, context_embeddings).max().item()  # Берем максимальное сходство

    # Штраф за неожиданные слова (учитываем редкость)
    unexpected_words = output_keywords - context_keywords
    unexpected_penalty = min(len(unexpected_words) / (len(output_keywords) + 1e-6), 1.0)

    # Итоговый коэффициент (BLEU + семантика + пересечение - штраф)
    final_score = (0.3 * semantic_similarity + 0.55 * bleu_mean + 0.15 * keyword_overlap) * (1 - unexpected_penalty)

    return max(final_score, 0)  # Исключаем отрицательные значения


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [40]:
# Пример использования
if __name__ == "__main__":
    # Пример данных
    contexts = [
        "Для оформления материальной помощи на оплату общежития необходимо предоставить определенные медицинские документы, такие как справка по форме 086у или сертификат МОДФ.",
        "Также нужно учесть, что условия предоставления мест и размещения в общежитиях различаются."
         "Более подробную информацию можно получить в Дирекции по управлению общежитиями, гостиницами, учебно-оздоровительными комплексами."
    ]
    model_output_good = "Для оформления материальной помощи на оплату общежития необходимо предоставить определенные медицинские документы"
    model_output_bad = "Интернет изобрел Илон Маск в 2010 году в гараже."

    # Проверка хорошего ответа
    good_score = hallucination_score(contexts, model_output_good)
    print(f"Score для хорошего ответа: {good_score:.4f}")

    # Проверка плохого (галлюцинирующего) ответа
    bad_score = hallucination_score(contexts, model_output_bad)
    print(f"Score для плохого ответа: {bad_score:.4f}")

Score для хорошего ответа: 0.4406
Score для плохого ответа: 0.0000


In [41]:
# Number 3.
# Пример 1
model_output_1 = "Для продления социальной стипендии необходимо подать заявку через сервис единого окна в модуле LMS, прикрепив отсканированные документы (личное заявление и документ, подтверждающий льготу)."
context_1 = ["Дополнительные документы для продления социальной стипендии нужно предоставить в Центр стипендиальных и благотворительных программ НИУ ВШЭ. Это включает копию заявления и копию действующей справки МСЭ. Для продления социальной стипендии необходимо подать заявку через сервис единого окна в модуле LMS, прикрепив отсканированные документы (личное заявление и документ, подтверждающий льготу)."]
# Проверка хорошего ответа
score = hallucination_score(context_1, model_output_1)
print(f"Score для хорошего ответа: {score:.4f}")

model_output_2 = "Для продления социальной стипендии необходимо подать заявку через сервис единого окна в модуле SMART LMS, прикрепив отсканированные документы (Паспорт и водительские права)."
# Проверка плохого ответа
score = hallucination_score(context_1, model_output_2)
print(f"Score для хорошего ответа: {score:.4f}")

Score для хорошего ответа: 0.6023
Score для хорошего ответа: 0.3544


4. Проверка противоречий между контекстами и ответом
Идея: Если model_output противоречит хотя бы одному из contexts (например, разные даты или имена), это признак галлюцинации. Можно использовать простую эвристику или более сложные методы (например, NLI).

In [21]:
!python -m spacy download ru_core_news_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 80.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.8/53.8 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 51.8 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('ru_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [23]:
import torch
import numpy as np
import re
import spacy
from sentence_transformers import SentenceTransformer, util
from transformers import GPT2LMHeadModel, GPT2Tokenizer

class FactVerificationScore:
    def __init__(self):
        self.sent_model = SentenceTransformer('all-MiniLM-L6-v2')
        self.gpt_tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
        self.gpt_model = GPT2LMHeadModel.from_pretrained('gpt2')
        self.nlp = spacy.load("ru_core_news_sm")  # Для русского языка
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        self.gpt_model.eval()
        self.gpt_model.to(self.device)

    def _encode_text(self, text):
        return self.sent_model.encode(text, convert_to_tensor=True, device=self.device)

    def _extract_facts(self, text):
        doc = self.nlp(text)
        entities = set(ent.text for ent in doc.ents)
        entities.update(re.findall(r'\b\d{4}\b|\b[А-ЯA-Z][а-яa-z]+(?: [А-ЯA-Z][а-яa-z]+)*', text))
        return entities

    def _calculate_perplexity(self, text):
        inputs = self.gpt_tokenizer(text, return_tensors="pt").to(self.device)
        with torch.no_grad():
            outputs = self.gpt_model(**inputs, labels=inputs["input_ids"])
            loss = outputs.loss
            perplexity = torch.exp(loss).item()
        return perplexity

    def _detect_negation(self, context):
        """ Проверяет, есть ли явное отрицание в контексте """
        negation_patterns = [
            r"информация.*отсутствует",
            r"не указано",
            r"нет данных",
            r"невозможно"
        ]
        return any(re.search(pattern, context.lower()) for pattern in negation_patterns)

    def calculate_score(self, model_output, context):
        # Концептуальная согласованность
        concept_score = util.cos_sim(self._encode_text(model_output), self._encode_text(context)).item()

        # Перплексия
        perplexity = self._calculate_perplexity(model_output)
        normalized_perplexity = min(1.0, perplexity / 50.0)

        # Фактологическая проверка
        model_facts = self._extract_facts(model_output)
        context_facts = self._extract_facts(context)
        hallucinated_facts = len(model_facts - context_facts)  # Новые факты, не из контекста
        missing_facts = len(context_facts - model_facts)       # Пропущенные факты из контекста
        fact_error_ratio = hallucinated_facts / max(1, len(model_facts))
        missing_fact_ratio = missing_facts / max(1, len(context_facts))

        # Проверка на явное отрицание в контексте
        negation_detected = self._detect_negation(context)
        negation_penalty = 1.0 if (negation_detected and hallucinated_facts > 0) else 0.0

        # Итоговый скор
        w_concept = 0.35       # Концептуальная согласованность
        w_perplexity = 0.15    # Перплексия (меньший вес, так как она менее значима)
        w_fact_error = 0.35    # Ошибки фактов
        w_missing = 0.15       # Пропущенные факты
        hallucination_score = (
            w_concept * (1 - concept_score) +
            w_perplexity * normalized_perplexity +
            w_fact_error * fact_error_ratio +
            w_missing * missing_fact_ratio +
            negation_penalty * 0.5  # Дополнительный штраф за противоречие отрицанию
        )
        hallucination_score = min(1, max(0, hallucination_score))

        return {
            "hallucination_score": hallucination_score,
            "concept_score": concept_score,
            "perplexity": perplexity,
            "fact_error_ratio": fact_error_ratio,
            "missing_fact_ratio": missing_fact_ratio,
            "hallucinated_facts": hallucinated_facts,
            "missing_facts": missing_facts,
            "negation_detected": negation_detected
        }

In [24]:
# Тестирование
verifier = FactVerificationScore()

# Пример 1
model_output_1 = "Для продления социальной стипендии необходимо подать заявку через сервис единого окна в модуле LMS, прикрепив отсканированные документы (личное заявление и документ, подтверждающий льготу)."
context_1 = "Дополнительные документы для продления социальной стипендии нужно предоставить в Центр стипендиальных и благотворительных программ НИУ ВШЭ. Это включает копию заявления и копию действующей справки МСЭ. Для продления социальной стипендии необходимо подать заявку через сервис единого окна в модуле LMS, прикрепив отсканированные документы (личное заявление и документ, подтверждающий льготу)."
result_1 = verifier.calculate_score(model_output_1, context_1)
print("Пример 1:", result_1)

# Пример 2
model_output_2 = "Получение автомата возможно, если студент выполняет все необходимые требования и проходит защиту ВКР успешно."
context_2 = "Информация о возможности получения автомата в данном тексте отсутствует."
result_2 = verifier.calculate_score(model_output_2, context_2)
print("Пример 2:", result_2)

Пример 1: {'hallucination_score': 0.28958986440726686, 'concept_score': 0.6217824816703796, 'perplexity': 9.54743480682373, 'fact_error_ratio': 0.0, 'missing_fact_ratio': 0.8571428571428571, 'hallucinated_facts': 0, 'missing_facts': 6, 'negation_detected': False}
Пример 2: {'hallucination_score': 1, 'concept_score': 0.7931008338928223, 'perplexity': 8.530292510986328, 'fact_error_ratio': 1.0, 'missing_fact_ratio': 1.0, 'hallucinated_facts': 2, 'missing_facts': 1, 'negation_detected': True}


In [29]:
verifier = FactVerificationScore()

# Пример 1
model_output_1 = "Для продления социальной стипендии необходимо подать заявку через сервис единого окна в модуле LMS, прикрепив отсканированные документы (личное заявление и документ, подтверждающий льготу)."
context_1 = "Дополнительные документы для продления социальной стипендии нужно предоставить в Центр стипендиальных и благотворительных программ НИУ ВШЭ. Это включает копию заявления и копию действующей справки МСЭ. Для продления социальной стипендии необходимо подать заявку через сервис единого окна в модуле LMS, прикрепив отсканированные документы (личное заявление и документ, подтверждающий льготу)."
result_1 = verifier.calculate_score(model_output_1, context_1)
print("Пример 1:", result_1)

model_output__2 = "Для оформления материальной помощи на оплату общежития необходимо предоставить определенные медицинские документы"
context_2 = "Для оформления материальной помощи на оплату общежития необходимо предоставить определенные медицинские документы, такие как справка по форме 086у или сертификат МОДФ.\
Также нужно учесть, что условия предоставления мест и размещения в общежитиях различаются.\
Более подробную информацию можно получить в Дирекции по управлению общежитиями, гостиницами, учебно-оздоровительными комплексами."
result_2 = verifier.calculate_score(model_output_2, context_2)
print("Пример 1:", result_2)

Пример 1: {'hallucination_score': 0.28958986440726686, 'concept_score': 0.6217824816703796, 'perplexity': 9.54743480682373, 'fact_error_ratio': 0.0, 'missing_fact_ratio': 0.8571428571428571, 'hallucinated_facts': 0, 'missing_facts': 6, 'negation_detected': False}
Пример 1: {'hallucination_score': 0.7097788798809052, 'concept_score': 0.4737485647201538, 'perplexity': 8.530292510986328, 'fact_error_ratio': 1.0, 'missing_fact_ratio': 1.0, 'hallucinated_facts': 2, 'missing_facts': 6, 'negation_detected': False}


# Математическая модель метрики FactVerificationScore

## Обозначения
Пусть:
- $O$ — текст ответа модели (`model_output`),
- $C$ — текст контекста (`context`),
- $S_{\text{concept}}(O, C)$ — концептуальная согласованность (косинусное сходство между $O$ и $C$),
- $P(O)$ — перплексия текста $O$,
- $F_O$ — множество фактов, извлечённых из $O$,
- $F_C$ — множество фактов, извлечённых из $C$,
- $H = |F_O \setminus F_C|$ — число "галлюцинированных" фактов,
- $M = |F_C \setminus F_O|$ — число "пропущенных" фактов,
- $R_{\text{fact}}$ — доля ошибочных (галлюцинированных) фактов,
- $R_{\text{missing}}$ — доля пропущенных фактов,
- $N(C)$ — индикатор наличия отрицания в контексте ($1$, если есть; $0$, если нет),
- $P_N$ — штраф за противоречие отрицанию,
- $w_{\text{concept}}, w_{\text{perplexity}}, w_{\text{fact}}, w_{\text{missing}}$ — весовые коэффициенты,
- $H_{\text{score}}$ — итоговый показатель галлюцинации.

## Компоненты метрики

### Концептуальная согласованность
Концептуальная согласованность:
$$ S_{\text{concept}}(O, C) = \cos(\text{Emb}(O), \text{Emb}(C)), $$
где $\text{Emb}(O)$ и $\text{Emb}(C)$ — векторные представления, полученные через `SentenceTransformer`.  
Для метрики используется:
$$ 1 - S_{\text{concept}}(O, C). $$

### Нормализованная перплексия
Перплексия:
$$ P(O) = e^{L(O)}, $$
где $L(O)$ — кросс-энтропийная потеря модели GPT-2. Нормализованная перплексия:
$$ P_{\text{norm}}(O) = \min\left(1, \frac{P(O)}{50}\right). $$

### Фактологическая проверка
- Число галлюцинированных фактов: $H = |F_O \setminus F_C|$,
- Число пропущенных фактов: $M = |F_C \setminus F_O|$,
- Доля ошибочных фактов:
$$ R_{\text{fact}} = \frac{H}{\max(1, |F_O|)}, $$
- Доля пропущенных фактов:
$$ R_{\text{missing}} = \frac{M}{\max(1, |F_C|)}. $$

### Штраф за отрицание
Индикатор отрицания:
$$ N(C) = \begin{cases}
1, & \text{если в } C \text{ найдено отрицание}, \\
0, & \text{иначе}.
\end{cases} $$
Штраф:
$$ P_N = \begin{cases}
1, & \text{если } N(C) = 1 \text{ и } H > 0, \\
0, & \text{иначе}.
\end{cases} $$
Вклад штрафа:
$$ \text{Negation Penalty} = P_N \cdot 0.5. $$

### Итоговый показатель
$$ H_{\text{score}} = \min\left(1, \max\left(0, w_{\text{concept}} \cdot (1 - S_{\text{concept}}) + w_{\text{perplexity}} \cdot P_{\text{norm}} + w_{\text{fact}} \cdot R_{\text{fact}} + w_{\text{missing}} \cdot R_{\text{missing}} + 0.5 \cdot P_N\right)\right), $$
где:
- $w_{\text{concept}} = 0.35$,
- $w_{\text{perplexity}} = 0.15$,
- $w_{\text{fact}} = 0.35$,
- $w_{\text{missing}} = 0.15$.

## Полная модель
$$ H_{\text{score}} = \min\left(1, \max\left(0, 0.35 \cdot (1 - \cos(\text{Emb}(O), \text{Emb}(C))) + 0.15 \cdot \min\left(1, \frac{e^{L(O)}}{50}\right) + 0.35 \cdot \frac{|F_O \setminus F_C|}{\max(1, |F_O|)} + 0.15 \cdot \frac{|F_C \setminus F_O|}{\max(1, |F_C|)} + 0.5 \cdot P_N\right)\right). $$

## Интерпретация
- $H_{\text{score}} \approx 0$: ответ полностью соответствует контексту.
- $H_{\text{score}} \approx 1$: ответ содержит значительные галлюцинации или противоречия.

## Пример применения

### Пример 1
- $S_{\text{concept}} \approx 0.98$,
- $P(O) \approx 12.34$, $P_{\text{norm}} = 0.247$,
- $H = 0$, $R_{\text{fact}} = 0$,
- $M = 3$, $|F_C| = 5$, $R_{\text{missing}} = 0.6$,
- $N(C) = 0$, $P_N = 0$.

$$ H_{\text{score}} = 0.35 \cdot 0.02 + 0.15 \cdot 0.247 + 0.35 \cdot 0 + 0.15 \cdot 0.6 = 0.134. $$

### Пример 2
- $S_{\text{concept}} \approx 0.45$,
- $P(O) \approx 15.67$, $P_{\text{norm}} = 0.313$,
- $H = 2$, $|F_O| = 2$, $R_{\text{fact}} = 1.0$,
- $M = 0$, $R_{\text{missing}} = 0$,
- $N(C) = 1$, $P_N = 1$.

$$ H_{\text{score}} = 0.35 \cdot 0.55 + 0.15 \cdot 0.313 + 0.35 \cdot 1.0 + 0.15 \cdot 0 + 0.5 \cdot 1 = 1.0895, $$
$$ H_{\text{score}} = \min(1, 1.0895) = 1. $$